In [2]:
from inside_ns import enter_ns
from orbit3d import get_3dtrajectory
import numpy as np

In [3]:
rin, vin = np.array([1.e20, 6e3, 0.]), np.array([-200., 0., 0.])
k = 1.32712e11

In [15]:
theta_ns, r, v, t = enter_ns(rin, vin, k, 10.)

/home/gsalinas/GitHub/orbits/orbit3d.py:17: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rotation, _ = Rotation.align_vectors(np.array([rin, vin]), np.array([r2d, v2d]))


In [18]:
theta0, rs, *_  = get_3dtrajectory(rin, vin, k)
theta02, rs2, *_  = get_3dtrajectory(r, v, k)

/home/gsalinas/GitHub/orbits/orbit3d.py:17: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rotation, _ = Rotation.align_vectors(np.array([rin, vin]), np.array([r2d, v2d]))


In [19]:
theta0, theta02

(mpf('3.1397842282901385739261961423882449574682266230649575'),
 mpc(real='2.3987732975628954617870552183119220091342501148990583', imag='0.42163423077071230454851111536166830133793699903117631'))